# Slowly Changing Dimensions made Easy with Durable Keys - Demo 1B: Usage Comparison - SCD Type 7

This series of demo's walks through how we can add a durable key to an existing dimension, how we adjust our merge statements to keep it up to date, and how we add that durable key into the fact table to allow easy As-posted, As-is and As-was queries.

- As-is – Shows data using current structure
- As-posted – Shows data using the “historical truth”, eg. the structure at the time that data point happened
- As-of – Shows data using structure at point in time, eg. 1st Jan 2020


***Note***: *None of the tables are indexed as they are so small, index as appropriate in your systems!*

## This Notebook contains the queries to compare queries with and without durable keys. This notebook has the queries without durable queries, and can be compared split screen with the notebook '/Demos/Demo 1A- Usage Comparison - SCD Type 2.ipynb'

In [1]:
-- If the Db got rebuilt, the connection is reset, and this will error without breaking the rest of the notebook!
USE DurableKeyDemo
GO

## Set the scene
This query aggregates sales using Product and Customer categories. However, these have changed in the course of 2020, so looking at the as-posted view breaks the data out into a bunch of catagories that overlap.

In [5]:
USE DurableKeyDemo
GO

SELECT ISNULL(ch.CustomerCategory, 'Total') AS CustomerCategory,
       ISNULL(ph.ProductCategory, 'Total') AS ProductCategory,
       SUM(s.Quantity * s.PricePaidPerUnit) AS SalesTotal,
       SUM(s.Quantity * (ph.ProductPrice - s.PricePaidPerUnit)) AS DiscountTotalAmount
FROM fact.Sales AS s
INNER JOIN dim.Calendar AS od
  ON od.CalendarSK = OrderDateSK
INNER JOIN dim.Product AS ph
  ON ph.ProductSK = s.ProductSK
INNER JOIN dim.Customer AS ch
  ON ch.CustomerSK = s.CustomerSK
WHERE od.CalendarYear = 2020
GROUP BY ch.CustomerCategory, ph.ProductCategory
  WITH ROLLUP
ORDER BY GROUPING(ch.CustomerCategory), ch.CustomerCategory, GROUPING(ph.ProductCategory), ph.ProductCategory


## Comparison Query 1
This query aggregates the tea sales by Customer and Product categories. In order to get the As-is Cstomer and Product categories, we have joined to the views for the current records using the durable key. This removes the joins using the business keys and simplifies the query, removing 1 join. For real world queries this can remove a number of joins

For the products we also need the as-posted price information, so we join to the Product dimnesion using the surrogate key.



In [4]:
USE DurableKeyDemo

SELECT ISNULL(cc.CurrentCustomerCategory, 'Total') AS CurrentCustomerCategory,
       ISNULL(pc.CurrentProductCategory, 'Total') AS CurrentProductCategory,
       SUM(s.Quantity * s.PricePaidPerUnit) AS SalesTotal,
       SUM(s.Quantity * (ph.ProductPrice - s.PricePaidPerUnit)) AS DiscountTotalAmount
FROM fact.Sales AS s
INNER JOIN dim.Calendar AS od
  ON od.CalendarSK = OrderDateSK

-- Join to Product for the hisorical As-posted price
INNER JOIN dim.Product AS ph
  ON ph.ProductSK = s.ProductSK

-- Join to ProductCurrent for the current As-is category
INNER JOIN dim.ProductCurrent AS pc
  ON pc.ProductDK = s.ProductDK


-- Join to CustomerCurrent for the current As-is data
INNER JOIN dim.CustomerCurrent AS cc
  ON cc.CustomerDK = s.CustomerDK






WHERE od.CalendarYear = 2020
GROUP BY cc.CurrentCustomerCategory, pc.CurrentProductCategory
  WITH ROLLUP
ORDER BY GROUPING(cc.CurrentCustomerCategory), cc.CurrentCustomerCategory, GROUPING(pc.CurrentProductCategory), pc.CurrentProductCategory


(9 rows affected)

Total execution time: 00:00:00.020

CurrentCustomerCategory,CurrentProductCategory,SalesTotal,DiscountTotalAmount
Advocate,Loose Leaf Tea,65.9500,0.0000
Advocate,Teabags,4.4900,0.5000
Advocate,Total,70.4400,0.5000
Irregular Customer,Teabags,3.5900,0.9000
Irregular Customer,Total,3.5900,0.9000
Trade - Big Fish,Loose Leaf Tea,6461.9700,356.8000
Trade - Big Fish,Teabags,66.8600,3.5000
Trade - Big Fish,Total,6528.8300,360.3000
Total,Total,6602.8600,361.7000


## Comparison Query 2
This query is the same as above, but instead of using the current version of the dimensions we get them as-was the end of May 2020. The joins now filter for the dimensional record that was valid at that time.

The query is simplified, joining to the dimensions directly with the durable key and filtering to the date range required, without the self join.

In [5]:
USE DurableKeyDemo

SELECT ISNULL(cc.CustomerCategory, 'Total') AS MayCustomerCategory,
       ISNULL(pc.ProductCategory, 'Total') AS MayProductCategory,
       SUM(s.Quantity * s.PricePaidPerUnit) AS SalesTotal,
       SUM(s.Quantity * (ph.ProductPrice - s.PricePaidPerUnit)) AS DiscountTotalAmount
FROM fact.Sales AS s
INNER JOIN dim.Calendar AS od
  ON od.CalendarSK = OrderDateSK

-- Join to Product for the hisorical As-posted price
INNER JOIN dim.Product AS ph
  ON ph.ProductSK = s.ProductSK

-- Join to ProductCurrent for the current As-is category
INNER JOIN dim.Product AS pc
  ON pc.ProductDK = s.ProductDK
     AND '20200531' BETWEEN pc.ValidFromDate and pc.ValidToDate


-- Join to CustomerCurrent for the current As-is data
INNER JOIN dim.Customer AS cc
  ON cc.CustomerDK = s.CustomerDK
     AND '20200531' BETWEEN cc.ValidFromDate and cc.ValidToDate




WHERE od.CalendarYear = 2020
GROUP BY cc.CustomerCategory, pc.ProductCategory
  WITH ROLLUP
ORDER BY GROUPING(cc.CustomerCategory), cc.CustomerCategory, GROUPING(pc.ProductCategory), pc.ProductCategory


(7 rows affected)

Total execution time: 00:00:00.021

MayCustomerCategory,MayProductCategory,SalesTotal,DiscountTotalAmount
Irregular Customer,Tea,3.5900,0.9000
Irregular Customer,Total,3.5900,0.9000
Super Customer,Tea,70.4400,0.5000
Super Customer,Total,70.4400,0.5000
Trade - Big Fish,Tea,6528.8300,360.3000
Trade - Big Fish,Total,6528.8300,360.3000
Total,Total,6602.8600,361.7000
